In [7]:
from google.colab import drive
drive.mount('/content/drive')

DRIVE_PATH = "/content/drive/MyDrive/YandexCup/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Загружаем скачанный классификатор токсичности:

#Score

In [8]:
!pip install -q transformers
!pip install -q natasha
!pip install -q pymorphy2
!pip install -q -U pymorphy2-dicts-ru
!pip install -q hnswlib
!pip install -q pymystem3
!pip install -q pymorphy2
!pip install -q -U pymorphy2-dicts-ru

In [9]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2021-10-14 15:55:23--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.241, 5.45.205.243, 5.45.205.245, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.241|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-man01i.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2021-10-14 15:55:24--  http://cache-man01i.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-man01i.cdn.yandex.net (cache-man01i.cdn.yandex.net)... 5.45.205.221, 2a02:6b8::3:221
Connecting to cache-man01i.cdn.yandex.net (cache-man01i.cdn.yandex.net)|5.45.205.221|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.2’

mystem-3.0-linux3.1 100%[===================>]  15.70M 

In [10]:
import gensim
from pymystem3 import Mystem

stemmer = Mystem()
lemmas = stemmer.lemmatize("Красивая мама красиво мыла раму")
print(lemmas)

['красивый', ' ', 'мама', ' ', 'красиво', ' ', 'мыть', ' ', 'рама', '\n']


In [11]:
!pip install https://github.com/kpu/kenlm/archive/master.zip
!pip install -q kenlm

     |████████████████████████████████| 540 kB 922 kB/s 
  Created wheel for kenlm: filename=kenlm-0.0.0-cp37-cp37m-linux_x86_64.whl size=2337417 sha256=bf4997c1223fe9493400dc09f4503133ca1dff9408513abda8da98729294a53b
  Stored in directory: /tmp/pip-ephem-wheel-cache-oo9ugm59/wheels/3d/aa/02/7b4a2eab5d7a2a9391bd9680dbad6270808a147bc3b7047e4e
Successfully built kenlm


In [12]:
import numpy as np

import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch import softmax, sigmoid
from tqdm.auto import tqdm
import argparse
from pymystem3 import Mystem
import sys
from collections import Counter
from functools import partial
import kenlm

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/YandexCup/trained_roberta")

model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/YandexCup/trained_roberta").cuda()

lm = kenlm.Model('/content/drive/MyDrive/YandexCup/lm.binary')



TOXIC_CLASS=-1
TOKENIZATION_TYPE='sentencepiece'


ALLOWED_ALPHABET=list(map(chr, range(ord('а'), ord('я') + 1)))
ALLOWED_ALPHABET.extend(map(chr, range(ord('a'), ord('z') + 1)))
ALLOWED_ALPHABET.extend(list(map(str.upper, ALLOWED_ALPHABET)))
ALLOWED_ALPHABET = set(ALLOWED_ALPHABET)


def logits_to_toxic_probas(logits):
    if logits.shape[-1] > 1:
        activation = lambda x: softmax(x, -1)
    else:
        activation = sigmoid
    return activation(logits)[:, TOXIC_CLASS].cpu().detach().numpy()


def is_word_start(token):
    if TOKENIZATION_TYPE == 'sentencepiece':
        return token.startswith('▁')
    if TOKENIZATION_TYPE == 'bert':
        return not token.startswith('##')
    raise ValueError("Unknown tokenization type")


def normalize(sentence, max_tokens_per_word=20):
    def validate_char(c):
        return c in ALLOWED_ALPHABET
    
    sentence = ''.join(map(lambda c: c if validate_char(c) else ' ', sentence.lower()))
    ids = tokenizer(sentence)['input_ids']
    tokens = tokenizer.convert_ids_to_tokens(ids)[1:-1]
    
    result = []
    num_continuation_tokens = 0
    for token in tokens:
        if not is_word_start(token):
            num_continuation_tokens += 1
            if num_continuation_tokens < max_tokens_per_word:
                result.append(token.lstrip('#▁'))
        else:
            num_continuation_tokens = 0
            result.extend([' ', token.lstrip('▁#')])
    
    return ''.join(result).strip()


def iterate_batches(data, batch_size=40):
    batch = []
    for x in data:
        batch.append(x)
        if len(batch) >= batch_size:
            yield batch
            batch = []
    if len(batch) > 0:
        yield batch


def predict_toxicity_1(sentences, batch_size=5, threshold=0.5, return_scores=False, verbose=True, device='cuda'):
    results = []
    for batch in iterate_batches(sentences, batch_size):
        normlized = [normalize(sent, max_tokens_per_word=5) for sent in batch]
        tokenized = tokenizer(normlized, return_tensors='pt', padding=True, max_length=512, truncation=True)
    
        logits = model.to(device)(**{key: val.to(device) for key, val in tokenized.items()}).logits
        preds = logits_to_toxic_probas(logits)
        if not return_scores:
            preds = preds >= threshold
        results.extend(preds)
    return results


def get_w2v_indicies_1(a):
    res = []
    if isinstance(a, str):
        a = a.split()
    for w in a:
        if w in embs_voc:
            res.append((w, embs_voc[w]))
        else:
            if w.isalpha():
                lemma = stemmer.lemmatize(w)[0]
                res.append((embs_voc.get(lemma), None))
    return res


def load_embeddings(path):
    embs_file = np.load(path, allow_pickle=True)
    embs_vectors = embs_file['vectors']
    embs_voc = embs_file['voc'].item()

    embs_voc_by_id = [None for i in range(len(embs_vectors))]
    for word, idx in embs_voc.items():
        if embs_voc_by_id[idx] is None:
            embs_voc_by_id[idx] = word
    return embs_vectors, embs_voc, embs_voc_by_id


def calc_embs_1(words):
    words = ' '.join(map(normalize, words))
    inds = get_w2v_indicies_1(words)
    return [(w, i if i is None else embs_vectors[i]) for w, i in inds]


def calc_single_embedding_dist(a, b):
    a_s, a_v = a  #слово, вектор
    b_s, b_v = b  #слово, вектор
    if a_s == b_s: #если слова равны
        return 0.0
    if a_v is None or b_v is None: #если одно из векторов нет - дать пизды 
        return 1.0
    a = a_v
    b = b_v
    # inexact match is punished by 0.1
    return 0.1 + 0.9 * (1 - a.dot(b) / np.linalg.norm(a) / np.linalg.norm(b)) / 2


def greedy_match_embs(a, b, max_dist=99999, cache=None, a_ind=0, b_ind=0):
    a_len = len(a) - a_ind #сколько слов мы еще не рассмотрели в а
    b_len = len(b) - b_ind #сколько слов мы еще не рассмотрели в b
    minlen = min(a_len, b_len) #минимальная длина строки
    maxlen = max(a_len, b_len) #минимальная длина строки
    if minlen == 0: 
        return np.minimum(maxlen, max_dist) 
    if maxlen - minlen >= max_dist: 
        return max_dist 
    
    if cache is None:
        cache = {}
    
    cache_key = (a_len, b_len)
    if cache_key in cache:
        return cache[cache_key]
        
    min_dist = max_dist
    
    first_dist = calc_single_embedding_dist(a[a_ind], b[b_ind])
    if max_dist >= first_dist:
        min_dist = np.minimum(min_dist, first_dist + greedy_match_embs(
            a, b, max_dist, cache, a_ind + 1, b_ind + 1
        ))
    
    if first_dist > 0 and max_dist >= 1:
        min_dist = np.minimum(min_dist, 1 + greedy_match_embs(
            a, b, max_dist, cache, a_ind + 1, b_ind
        ))
        min_dist = np.minimum(min_dist, 1 + greedy_match_embs(
            a, b, max_dist, cache, a_ind, b_ind + 1
        ))
    
    cache[cache_key] = min_dist
    
    return min_dist



def calc_semantic_distance(a, b):
    a_embs = calc_embs_1(a)
    b_embs = calc_embs_1(b)
    
    clip_distance = 5  # this clips long computations
    return np.exp(-(greedy_match_embs(a_embs, b_embs, max_dist=clip_distance) / (0.6 * np.log(1 + len(a)))) ** 2)


def distance_score(original, fixed):
    original = original.split()
    fixed = fixed.split()
    
    return calc_semantic_distance(original, fixed)


def compute_lmdiff(original, fixed):
    original_lm_logproba = lm.score(original, bos=True, eos=True)
    fixed_lm_logproba = lm.score(fixed, bos=True, eos=True)
    
    probability_fraction = 10**((fixed_lm_logproba - original_lm_logproba) / 25)
    
    return np.clip(probability_fraction, 0.0, 1.0)


def compute_score(original_sentences, fixed_sentences, threshold=0.5, batch_size=5, show_res = False, return_result = False):
    fixed_toxicities = predict_toxicity_1(fixed_sentences, threshold=threshold, batch_size=batch_size, return_scores=True)
    scores = []
    lmdiffs = []
    emb_dists = []
    for original_sentence, fixed_sentence, fixed_toxicity in tqdm(zip(
        original_sentences, fixed_sentences, fixed_toxicities
    ), miniters=250, disable= True):
        original_sentence = normalize(original_sentence)
        fixed_sentence = normalize(fixed_sentence)
        
        distance = distance_score(original_sentence, fixed_sentence)
        lmdiff = compute_lmdiff(original_sentence, fixed_sentence)
        
        score = (1 - fixed_toxicity) * distance * lmdiff
        
        lmdiffs.append(lmdiff)
        emb_dists.append(distance)
        scores.append(score)
        
    if show_res:
        print('average toxicity:', np.mean(fixed_toxicities), file=sys.stderr)
        print('mean lmdiff:', np.mean(lmdiffs), file=sys.stderr)
        print('mean distance_score:', np.mean(emb_dists), file=sys.stderr)
        print('score', np.mean(scores) * 100, file=sys.stderr)
        
    if return_result:
        return {"scores" : scores, 
                "toxicities" : fixed_toxicities, 
                "lmdiffs" : lmdiffs,
                "distance_scores" : emb_dists}
    else:
        return np.mean(scores)

'''
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('original_texts', type=argparse.FileType('r'))
    parser.add_argument('fixed_texts', type=argparse.FileType('r'))
    parser.add_argument('--score', type=argparse.FileType('w'))
    parser.add_argument('--model', required=True, type=str)
    parser.add_argument('--embeddings', type=str, required=True)
    parser.add_argument('--lm', type=str, required=True)
    parser.add_argument('--device', type=str, choices=['cuda', 'cpu'], default='cpu')
    
    return parser.parse_args()
'''

def new_checker(original_texts, fixed_texts, device='cuda', show_res = False, return_result = False):
    original_texts = list(map(str.strip, original_texts))
    fixed_texts = list(map(str.strip, fixed_texts))
    
    assert len(original_texts) == len(fixed_texts)
    
    with torch.inference_mode(True):
        if return_result:
            return compute_score(original_texts, fixed_texts, show_res = show_res, return_result=return_result)
        return (100 * compute_score(original_texts, fixed_texts, show_res = show_res))

#Solution:

In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained(DRIVE_PATH + "trained_roberta")

model = AutoModelForSequenceClassification.from_pretrained(DRIVE_PATH + "trained_roberta").cuda()

TOXIC_CLASS=-1
TOKENIZATION_TYPE='sentencepiece'


Ниже функции для применения классификатора

In [14]:
from torch import softmax, sigmoid
import numpy as np


import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from tqdm.auto import tqdm
import argparse
from pymystem3 import Mystem
import sys
from collections import Counter
from functools import partial



ALLOWED_ALPHABET=list(map(chr, range(ord('а'), ord('я') + 1)))
ALLOWED_ALPHABET.extend(map(chr, range(ord('a'), ord('z') + 1)))
ALLOWED_ALPHABET.extend(list(map(str.upper, ALLOWED_ALPHABET)))
ALLOWED_ALPHABET = set(ALLOWED_ALPHABET)


def logits_to_toxic_probas(logits):
    if logits.shape[-1] > 1:
        activation = lambda x: softmax(x, -1)
    else:
        activation = sigmoid
    return activation(logits)[:, TOXIC_CLASS].cpu().detach().numpy()


def is_word_start(token):
    if TOKENIZATION_TYPE == 'sentencepiece':
        return token.startswith('▁')
    if TOKENIZATION_TYPE == 'bert':
        return not token.startswith('##')
    raise ValueError("Unknown tokenization type")


def normalize(sentence, max_tokens_per_word=20):
    def validate_char(c):
        return c in ALLOWED_ALPHABET
    
    sentence = ''.join(map(lambda c: c if validate_char(c) else ' ', sentence.lower()))
    ids = tokenizer(sentence)['input_ids']
    tokens = tokenizer.convert_ids_to_tokens(ids)[1:-1]
    
    result = []
    num_continuation_tokens = 0
    for token in tokens:
        if not is_word_start(token):
            num_continuation_tokens += 1
            if num_continuation_tokens < max_tokens_per_word:
                result.append(token.lstrip('#▁'))
        else:
            num_continuation_tokens = 0
            result.extend([' ', token.lstrip('▁#')])
    
    return ''.join(result).strip()


def iterate_batches(data, batch_size=40):
    batch = []
    for x in data:
        batch.append(x)
        if len(batch) >= batch_size:
            yield batch
            batch = []
    if len(batch) > 0:
        yield batch

from tqdm.auto import tqdm
def predict_toxicity(sentences, batch_size=5, threshold=0.5, return_scores=False, verbose=True, device='cuda', off_tqdm=False):
    results = []
    tqdm_fn = tqdm if verbose else lambda x, total: x
    for batch in tqdm_fn(iterate_batches(sentences, batch_size), total=np.ceil(len(sentences) / batch_size),  disable=off_tqdm):
        normlized = [normalize(sent, max_tokens_per_word=5) for sent in batch]
        tokenized = tokenizer(normlized, return_tensors='pt', padding=True, max_length=512, truncation=True)
        
        model_input = {key: val.to(device) for key, val in tokenized.items()}
        
        logits = model.to(device)(**model_input).logits
        
        preds = logits_to_toxic_probas(logits)
        
        if not return_scores:
            preds = preds >= threshold
        results.extend(preds)
    return results


In [15]:
texts = []
raw_texts = []
with open(DRIVE_PATH + 'public_testset.txt', 'rt') as f:
    for line in f:
        raw_texts.append(line) 
        texts.append(normalize(line))

Token indices sequence length is longer than the specified maximum sequence length for this model (534 > 512). Running this sequence through the model will result in indexing errors


Вычисляем токсичность отдельных слов

In [16]:
line_toxicities = []
line_mean_toxicities = []

for line in tqdm(np.random.choice(raw_texts, 25)):
    pred = predict_toxicity([line], return_scores=True, off_tqdm=True)
    
    line_words = normalize(line).split()
    toxi = [predict_toxicity([word], return_scores=True, off_tqdm=True) for word in line_words]
    
    line_toxicities.append(pred[0])
    line_mean_toxicities.append(np.mean(toxi))



  0%|          | 0/25 [00:00<?, ?it/s]

In [17]:
print(f"На ({len(line_toxicities)} комментах):")
print(np.mean(line_toxicities), f"- средняя токсичность всего комментария")
print(np.mean(line_mean_toxicities), "- cредняя токсичность каждого слова в комментарии")

На (25 комментах):
0.73615175 - средняя токсичность всего комментария
0.20213725 - cредняя токсичность каждого слова в комментарии


In [18]:
line = raw_texts[100]
pred = predict_toxicity([line], return_scores=True, off_tqdm=True)

line_words = normalize(line).split()
toxi = [predict_toxicity([word], return_scores=True, off_tqdm=True) for word in line_words]

print(line)
print(pred)
print(line_words)
print(toxi)

Можно сказать проще - кукуха едет от старческого маразма.

[0.8303647]
['можно', 'сказать', 'проще', 'кукуха', 'едет', 'от', 'старческого', 'маразма']
[[0.07630243], [0.2619401], [0.04409236], [0.56395835], [0.15138271], [0.0645212], [0.2058756], [0.63614625]]


Ниже читаем эмбеддинги слов и описываем функции их обработки

In [19]:
!pip install -q pymorphy2
!pip install -q -U pymorphy2-dicts-ru

from pymystem3 import Mystem
from pymorphy2 import MorphAnalyzer
import pymorphy2


morph = pymorphy2.MorphAnalyzer(lang='ru')

In [20]:
def save_comments(comments, name):
    with open(DRIVE_PATH + f'submits/{name}.txt', 'wt') as f:
        for text in comments:
            print(text, file=f)
        

In [21]:
embs_file = np.load(DRIVE_PATH + 'embeddings_with_lemmas.npz', allow_pickle=True)
embs_vectors = embs_file['vectors']
embs_vectors_normed = embs_vectors / np.linalg.norm(embs_vectors, axis=1, keepdims=True)
embs_voc = embs_file['voc'].item()

embs_voc_by_id = [None for i in range(len(embs_vectors))]
for word, idx in embs_voc.items():
    if embs_voc_by_id[idx] is None:
        embs_voc_by_id[idx] = word

#Улучшаем словарь toxicity

In [66]:
toxicity = dict()

In [63]:
#словарь русских ругательств
def improve_toxicity(data):

    words = set()
    for text in data:
        words.update(text.split())
    words = sorted(words)

    with torch.inference_mode():
        word_toxicities = predict_toxicity(words, batch_size=100, return_scores=True)

    before = len(toxicity)
    for word, tox in zip(words, word_toxicities):
        toxicity[word] = tox
    print("New words :", len(toxicity) - before)
    print("Cur len :", len(toxicity))


In [67]:
#1) Эмбеддинги
data = embs_voc.keys()
improve_toxicity(data)

  0%|          | 0/1627.0 [00:00<?, ?it/s]

New words : 162674
Cur len : 162674


In [68]:
#2) Комментарии
data = texts
improve_toxicity(data)

  0%|          | 0/221.0 [00:00<?, ?it/s]

New words : 15015
Cur len : 177689


In [69]:
#3) Словарь ругательств
data = []
with open(DRIVE_PATH + 'dicitionary/russkiy-mat.txt', 'rt') as f:
    for line in f:
        line = normalize(line)
        if line:
            data.append(line)

improve_toxicity(data)

  0%|          | 0/170.0 [00:00<?, ?it/s]

New words : 9797
Cur len : 187486


In [70]:
#4) Комментарии из какого-то датасета
import pandas as pd
comments = pd.read_csv(DRIVE_PATH + "dicitionary/final.csv")

data = []
for line in tqdm(comments["text"].tolist()):
    line = normalize(line)
    if line:
        data.append(line)
        
improve_toxicity(data)

  0%|          | 0/248287 [00:00<?, ?it/s]

  0%|          | 0/2358.0 [00:00<?, ?it/s]

New words : 181041
Cur len : 368527


In [ ]:
sorted_toxicity = {k: v for k, v in sorted(toxicity.items(), key=lambda item: item[1], reverse = True)}
for k, v in list(sorted_toxicity.items())[-100:]:
    print(k ,v)

In [24]:
def get_w2v_indicies(a):
    res = []
    if isinstance(a, str):
        a = a.split()
    for w in a:
        if w in embs_voc:
            res.append(embs_voc[w])
        else:
            
            lemma = stemmer.lemmatize(w)[0]#morph.parse(w)[0].normal_form
            res.append(embs_voc.get(lemma, None))
    return res

def calc_embs(words):
    words = ' '.join(map(normalize, words))
    inds = get_w2v_indicies(words)

    return [None if i is None else embs_vectors[i] for i in inds]

Сложим эмбеддинги нетоксичных слов в kd-дерево, чтобы можно было близко искать ближайших соседей

In [26]:
MAX_TOXICITY  = 1

non_toxicity = {word : val for word, val in toxicity.items() if val <= MAX_TOXICITY}

nontoxic_emb_inds = [ind for word, ind in embs_voc.items() if toxicity.get(word, 1.0) <= MAX_TOXICITY]
embs_vectors_normed_nontoxic = embs_vectors_normed[nontoxic_emb_inds]

print(len(non_toxicity))

162690


In [27]:
#Super non toxic words 

MAX_TOXICITY  = 0.01

super_non_toxic = {word : val for word, val in toxicity.items() if val <= MAX_TOXICITY}

super_nontoxic_emb_inds = [ind for word, ind in embs_voc.items() if toxicity.get(word, 1.0) <= MAX_TOXICITY]
super_embs_vectors_normed_nontoxic = embs_vectors_normed[super_nontoxic_emb_inds]

print(len(super_non_toxic))
print(super_non_toxic)
print(len(super_embs_vectors_normed_nontoxic))

255
{'howo': 0.009838613, 'section': 0.009659221, 'selected': 0.009694491, '2-4mp': 0.009839244, 'result': 0.009991114, 'mpm': 0.0096359, 'postscript': 0.00974957, 'дизайн-проект': 0.009940755, 'ctp': 0.00979208, 'workbook': 0.00958096, 'product': 0.009963871, 'j-1': 0.009945445, 'apply': 0.009644574, 'certificate': 0.009878223, 'cardin': 0.009779644, '[text': 0.009994719, 'mp3': 0.009839244, 'projector': 0.009711683, 'rental': 0.009812759, 'mps': 0.009665829, 'ideaphone': 0.009980343, '-дизайн': 0.009810781, 'airflow': 0.009890847, 'pocketbook': 0.009745045, 'purchase': 0.009702743, 'checkpoint': 0.00965198, 'b-class': 0.009974619, 'information': 0.0099925855, 'registered': 0.009675798, 'vvv': 0.009896731, 'callback': 0.00972145, 'j7': 0.009945445, 'documents': 0.009758518, 'dcfta': 0.009765147, 'promotion': 0.009836448, 'polycom': 0.009978759, 'jcomments': 0.009724541, 'projects': 0.00969858, 'casein': 0.009815169, 'allview': 0.009727797, 'lifebook': 0.009661553, 'easy': 0.009900264,

Функция находит самое близкое нетоксичное слово по предпосчитанным эмбеддингам слов

In [28]:
embs_vectors_normed_nontoxic.shape

(162690, 300)

In [29]:
import hnswlib
import numpy as np
import pickle


num_elements, dim = super_embs_vectors_normed_nontoxic.shape

# Generating sample data
data = super_embs_vectors_normed_nontoxic
ids = np.arange(num_elements)

# Declaring index
p = hnswlib.Index(space = 'cosine', dim = dim) # possible options are l2, cosine or ip

# Initializing index - the maximum number of elements should be known beforehand
p.init_index(max_elements = num_elements, ef_construction = 400, M = 16)

# Element insertion (can be called several times):
p.add_items(data, ids)

In [30]:
#A better one

from functools import lru_cache

@lru_cache()
def find_closest_nontoxic(word, word_toxicity, allow_self=False):
    #if toxicity.get(word, 1.0) <= threshold:
    #    return word

    #word_lemmatized = morph.parse(word)[0].normal_form
    if word not in embs_voc:
        
            return word_toxicity, word
        #else:
           # word = word_lemmatized
    

    #threshold = min(toxicity.get(word, threshold), threshold)
    
    word_emb = calc_embs([word])
    if word_emb is None or word_emb[0] is None:
        return word_toxicity, word


    query = p.knn_query(word_emb, k = 200)[0][0]


    best_candidate = word
    best_score = word_toxicity

    

    for i in query:
        candidate = embs_voc_by_id[nontoxic_emb_inds[i]]
        similarity = distance_score(word, candidate)
        
        if (1 - toxicity.get(candidate)) + (similarity) > best_score:
            
            best_score = (1 - toxicity.get(candidate)) + (similarity)
            best_candidate = candidate 
            
    return toxicity.get(candidate), best_candidate


Заменяем токсичные слова на ближайшие по эмбеддингам не-токсичные

In [31]:
def sort_by_toxicity(words):
    toxicities = [toxicity.get(w) if w in toxicity else predict_toxicity([w], return_scores = True, off_tqdm=True) for w in words]
  
    result = [[toxicities[i], i, words[i]] for i in range(len(words))]
    result.sort()
    return result

line = line = np.random.choice(raw_texts)
pred = predict_toxicity([line], return_scores=True, off_tqdm=True)

line_words = normalize(line).split()

print(sort_by_toxicity(line_words))

[[0.03607923, 0, 'да'], [0.037498195, 16, 'же'], [0.05111447, 1, 'это'], [0.058970306, 12, 'опять'], [[0.06933392], 10, 'эти'], [0.07904618, 5, 'давно'], [0.08566516, 14, 'на'], [0.112177566, 7, 'в'], [0.14424363, 15, 'те'], [0.14814769, 4, 'люди'], [0.15841357, 9, 'но'], [[0.20971347], 13, 'наступают'], [[0.3590889], 3, 'умные'], [0.3685782, 17, 'грабли'], [[0.5255045], 6, 'выкинули'], [[0.5718526], 11, 'недоросли'], [0.59206426, 8, 'унитаз'], [0.86097527, 2, 'говно']]


In [32]:
def sort_by_distance(words, word):
    result = [[distance_score(word, words[i]), i, words[i]] for i in range(len(words))]
    result.sort()
    return result

print(texts[0])
sort_by_distance(texts[0].split(), "смерть")

он скоро сдохнет и все вернется


[[0.0030838375406632423, 2, 'сдохнет'],
 [0.0030838375406632423, 5, 'вернется'],
 [0.3301158710261462, 4, 'все'],
 [0.3321799419260301, 1, 'скоро'],
 [0.35243420745805787, 3, 'и'],
 [0.47781617663736303, 0, 'он']]

In [33]:
def detox(line):
    
    words = normalize(line).split()
    sorted_words = sort_by_toxicity(words) #[токсичность, индекс, слово]
    
    
    #cur_toxicity = predict_toxicity([' '.join(words)], return_scores=True, off_tqdm=True)[0]
  
    while sorted_words:
        toxic_toxicity, toxic_idx, toxic_word = sorted_words.pop()
        if toxic_toxicity > 0.77:
            fixed_word = "Спасибо"
        else:
            fixed_toxicity, fixed_word = find_closest_nontoxic(toxic_word, toxic_toxicity, allow_self = True)
        words[toxic_idx] = fixed_word
        #if fixed_word != toxic_word:
        #    sorted_words.append([fixed_toxicity, toxic_idx, fixed_word])
        #    sorted_words.sort()
        
        #cur_toxicity = predict_toxicity([' '.join(words)], return_scores=True, off_tqdm=True)[0]
        

    
    return ' '.join(words)

In [34]:
#Super non toxic
def find_most_similar(word):
    emb_word = calc_embs([word])
    max_similarity = 0
    result = "спасибо"
    for i in p.knn_query(emb_word, k = 5)[0][0]:
        
        other_word = embs_voc_by_id[super_nontoxic_emb_inds[i]]
        
        similarity = distance_score(word, other_word)
        if similarity > max_similarity:
            max_similarity = similarity
            result = other_word
        #print(other_word, similarity, toxicity.get(other_word))
    return result


def detox_dumb(line):
    words = normalize(line).split()
    sorted_words = sort_by_distance(words, "спасибо")#sort_by_toxicity(words)
    score = new_checker([line], [' '.join(words)])
    
    while sorted_words[::-1]:
        toxic_toxicity, toxic_idx, toxic_word = sorted_words.pop()
        
        #if toxic_word in embs_voc:
        #    fixed_word = find_most_similar(toxic_word)
        #else:
        fixed_word = "спасибо"
        
        

        words[toxic_idx] = fixed_word
        new_score = new_checker([line], [' '.join(words)])

        if new_score >= score:
            score = new_score
        else:
            words[toxic_idx] = toxic_word
            #break
    
    return ' '.join(words)

#scores = np.array(compute_score2([line for i in range(len(detoxed_sentences))], detoxed_sentences))
#        words = detoxed_sentences[np.argmax(scores)].split()

In [ ]:
Original = []
Detoxed = []
for i in tqdm(range(0, len(texts), 50)):
    line = texts[i]
    detoxed = detox_dumb(line)

    Original.append(line)
    Detoxed.append(detoxed)


new_checker(Original, Detoxed, show_res = True)

  0%|          | 0/50 [00:00<?, ?it/s]

average toxicity: 0.33889008
mean lmdiff: 0.9461361997563867
mean distance_score: 0.7246765191251217


46.035975132307

In [ ]:
predict_toxicity([w], return_scores = True)

  0%|          | 0/1.0 [00:00<?, ?it/s]

[0.24380866]

In [ ]:
fixed_texts = list(map(detox_dumb, tqdm(texts)))

save_comments(fixed_texts, "closest_to_thanks")

new_checker(texts, fixed_texts, show_res = True)

  0%|          | 0/2500 [00:00<?, ?it/s]

average toxicity: 0.34476024
mean lmdiff: 0.9481122214124138
mean distance_score: 0.7274946147156974


46.144001054329166

#Сравнение решений:

In [ ]:
valya = []
kirill = []
count_valya = 0
count_kirill = 0 

with open(DRIVE_PATH + 'submits/bruteforce_hard_2_part.txt', 'rt') as f:
    for line in f:
        valya.append(line) 

with open(DRIVE_PATH + 'submits/full_thanks.txt', 'rt') as f:
    for line in f:
        kirill.append(line) 

super_result = []
for i in tqdm(range(2500)):
    kirill_score =  new_checker([texts[i]], [kirill[i]])
    valya_score = new_checker([texts[i]], [valya[i]])
    if valya_score > kirill_score:
        count_valya += 1
        super_result.append(valya[i])
        #print(valya_score,kirill_score, "Валя")
    else:
        count_kirill += 1
        super_result.append(kirill[i])
        #print(valya_score, kirill_score, "Кирилл")

    

print(count_valya, count_kirill)
new_checker(texts, super_result, show_res = True)

  0%|          | 0/2500 [00:00<?, ?it/s]

686 1814


average toxicity: 0.31801412
mean lmdiff: 0.9771276972705748
mean distance_score: 0.755816421040236


51.008772758835654

#Merges solutions into ultimate one

In [ ]:
ultimate_solution = []

names = os.listdir(path="/content/drive/MyDrive/YandexCup/submits_pro")
if '.ipynb_checkpoints' in names:
    names.remove('.ipynb_checkpoints')


solutions = {name : [] for name in names}
count = {name : 0 for name in names}
#reading submits
for name in names:
    sol = []
    with open(DRIVE_PATH + f'submits_pro/{name}', 'rt') as f:
        for line in f:
            if line.strip():
                sol.append(normalize(line))

    solutions[name] = sol[:]
    print(name, len(sol))

for i in tqdm(range(len(texts))):
    scores = {name : 0 for name in names}
    for name in names:
        scores[name] = new_checker([texts[i]], [solutions[name][i]])

    best_sol_name = max(scores, key=scores.get)
    count[best_sol_name] += 1
    ultimate_solution.append(solutions[best_sol_name][i])


print(count)
print(new_checker(texts, ultimate_solution, show_res = True))
#save_comments(ultimate_solution, "mega")

Token indices sequence length is longer than the specified maximum sequence length for this model (534 > 512). Running this sequence through the model will result in indexing errors


super_2.txt 2500
cool.txt 2500


  0%|          | 0/2500 [00:00<?, ?it/s]

{'super_2.txt': 2178, 'cool.txt': 322}
53.2548640218932


average toxicity: 0.3053273
mean lmdiff: 0.9732874209663734
mean distance_score: 0.7775705304398256
score 53.2548640218932


In [ ]:
save_comments(ultimate_solution, "lalala")

# Сортируем по скору. Анализируем

In [ ]:
solution = []

with open(DRIVE_PATH + 'submits_pro/cool.txt', 'rt') as f:
    for line in f:
        solution.append(normalize(line))


result = new_checker(texts, solution, return_result = True)

In [ ]:
print(result)
print(np.mean(result["scores"]))

{'scores': [0.363118312358863, 0.4103537799650043, 0.5835331934211689, 0.389562726020813, 0.2030926493166641, 0.26071022383085407, 0.44466835556224255, 0.6107004874672373, 0.3759409517069033, 0.5176647229739805, 0.7115012779509674, 0.5133160948753357, 0.989504829980433, 0.34543814565741776, 0.5598297899992206, 0.3156755964736664, 0.2910840427172206, 0.34251797971646564, 0.4433456807239648, 0.3374713786534226, 0.2796941201102418, 0.4467968803573997, 0.050262894820802415, 0.9091440237460445, 0.8445682376623154, 0.7501256825133878, 0.4428281409016097, 0.5797071649522693, 0.37061311189881446, 0.8035510629415512, 0.6498369549598112, 0.31670269266969714, 0.3918898204006275, 0.21951766852611979, 0.3913344426137179, 0.7788709301022811, 0.36910870906870874, 0.47994950532146297, 0.3746118621947193, 0.45999213658619775, 0.7666857043922712, 0.5425011836513602, 0.592260867357254, 0.30798521350226965, 0.4287836474939742, 0.863673527343035, 0.7788893936422517, 0.7200039818790822, 0.5516834055114586, 

In [ ]:
scores = np.array(compute_score2([line for i in range(len(detoxed_sentences))], detoxed_sentences))
#        words = detoxed_sentences[np.argmax(scores)].split()

In [ ]:
import string


In [ ]:
best_score = new_checker([texts[2330]], [texts[2330]], show_res = True)
print(best_score)
new_word = ''
for w in tqdm(embs_voc.keys()):
    sc = new_checker([texts[2330]], [w])
    if best_score < sc:
        best_score = sc
        new_word = w
        print(new_word)
        print(best_score)
print(new_word)
print(best_score)

4.677951335906982


average toxicity: 0.9532205
mean lmdiff: 1.0
mean distance_score: 1.0
score 4.677951335906982


  0%|          | 0/162690 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
sorted_by_score = [[score, i, solution[i]] for i, score in enumerate(result["scores"])]
sorted_by_score.sort()
for score, i, text in sorted_by_score:
    print(i, score, text)

##Для Online-решения

In [ ]:
print(len(nontoxic_emb_inds))
print(len(toxicity))

162690
162690


In [76]:
import pickle as pkl

with open(DRIVE_PATH + 'data.pkl', 'wb') as f:
    pkl.dump(toxicity, f)
    

In [75]:
len(toxicity)

368527

Сохраним данные для бейзлайна online-задачи